### Setup

In [ ]:
import os

from histogram_client import HistogramClient
from tango import DeviceProxy

# Look up the external IP and port for tango-databaseds in your deployed namespace
os.environ["TANGO_HOST"] = "192.168.128.144:10000"

# Set to the talon board you want to check
talon = "001"

In [ ]:
import math
from time import sleep

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def plot_histogram(plot_device, plot_data):
    axis_range = np.linspace(-1, 1, 64)
    plot_fig = make_subplots(
        rows=2,
        cols=4,
        shared_xaxes=True,
        shared_yaxes=True,
        column_titles=("Pol-X", "Pol-X", "Pol-Y", "Pol-Y"),
        column_widths=[4, 1, 4, 1],
        row_heights=[4, 1],
        x_title="Real",
        y_title="Imaginary",
    )

    for idx, pol in {0: "X", 1: "Y"}.items():
        # histogram contour of samples per polarization
        pol_data = plot_data[idx]
        fig.add_trace(
            go.Contour(
                x=axis_range,
                y=axis_range,
                z=pol_data,
                name=f"Pol-{pol}",
            ),
            row=1,
            col=1 + idx * 2,
        )

        # sum all rows for real component
        real_counts = np.sum(pol_data, axis=0)
        plot_fig.add_trace(
            go.Bar(
                x=axis_range,
                y=real_counts,
            ),
            row=2,
            col=1 + idx * 2,
        )

        # sum all columns for imaginary component
        imag_counts = np.sum(pol_data, axis=1)
        plot_fig.add_trace(
            go.Bar(
                y=axis_range,
                x=imag_counts,
                orientation="h",
            ),
            row=1,
            col=2 + idx * 2,
        )

        # calculate power in dB
        samples = real_counts.sum()
        if samples > 0:
            power = (
                np.sum(
                    np.asarray(pol_data)
                    * np.power(
                        np.abs(axis_range + 1j * axis_range),
                        2,
                    )
                )
                / samples
            )
            power_db = 20 * np.log10(power)
        else:
            power_db = np.NaN
        print(f"{plot_device} pol-{pol}: {samples} samples, power = {power_db:.1f} dB")

    width = 1024
    plot_fig.update_layout(
        title=f"Histograms of complex signals from {plot_device}, channel 0",  # TODO add support for channel selection, default is 0
        width=width,
        height=width // 2,
        showlegend=False,
    )

    plot_fig.show()

### Wideband State Count

In [ ]:
# Settings


device = f"talondx-{talon}/wbstatecount/state_count"
n_bins = 4096
sample_accum = 100000

dp = DeviceProxy(device)

In [ ]:
dp.command_inout("state_count_reset")

dp.write_attribute("sample_accum", sample_accum)
dp.write_attribute("bin_number_select", n_bins)
dp.command_inout("state_count_capture")

In [ ]:
def read_vectors(read_dp):
    read_state_count_vec_1 = read_dp.read_attribute("state_count_vector_1").value
    read_state_count_vec_2 = read_dp.read_attribute("state_count_vector_2").value
    read_power_spectrum_1 = read_dp.read_attribute("psd_vector_1").value
    read_power_spectrum_2 = read_dp.read_attribute("psd_vector_2").value

    # state count vector and psd vector 3/4 are not used for band 1/2
    dp.command_inout("state_count_reset")
    return (
        read_state_count_vec_1,
        read_state_count_vec_2,
        read_power_spectrum_1,
        read_power_spectrum_2,
    )


state_count_vec_1, state_count_vec_2, power_spectrum_1, power_spectrum_2 = read_vectors(dp)

In [ ]:
x_axis = list(range(-math.ceil(n_bins / 2), math.floor(n_bins / 2) + 1))
psd_x = list(range(1, len(power_spectrum_1) + 1))
normalized_x_axis = [(x - min(x_axis)) / (max(x_axis) - min(x_axis)) * (2.0) - 1.0 for x in x_axis]
normalized_psd_x = [(x - min(psd_x)) / (max(psd_x) - min(psd_x)) * 1.0 for x in psd_x]

fig = make_subplots(
    rows=2,
    cols=2,
    vertical_spacing=0.2,
    subplot_titles=(
        "State Count (pol X/A sub-band 1)",
        "State Count (pol Y/B sub-band 1)",
        "Power Spectrum (Pol X/A sub-band 1)",
        "Power Spectrum (pol Y/B sub-band 1)",
    ),
)

fig["layout"]["plot_bgcolor"] = "#fafafa"
fig["layout"]["showlegend"] = False
fig.add_trace(go.Bar(x=normalized_x_axis, y=state_count_vec_1, name="state count 1"), row=1, col=1)
fig.add_trace(go.Bar(x=normalized_x_axis, y=state_count_vec_2, name="state count 2"), row=1, col=2)
fig.add_trace(
    go.Bar(x=normalized_psd_x, y=power_spectrum_1, name="power spectrum 1"), row=2, col=1
)
fig.add_trace(
    go.Bar(x=normalized_psd_x, y=power_spectrum_2, name="power spectrum 2"), row=2, col=2
)
fig.update_layout(title_text=f"{device} sub-band 1", height=700)
fig.show()

### Wideband Input Buffer 

In [ ]:
device = f"talondx-{talon}/wideband-input-buffer/wideband-input-buffer"

dp = DeviceProxy(device)

receptor_id = dp.read_attribute("DishID").value
rx_sample_rate = dp.read_attribute("RxSampleRate").value

meta_transport_sample_rate = dp.read_attribute("MetaTransportSampleRate").value

print(f"Receptor ID = {receptor_id}")

# the number of sample pairs received per PPS interval
print(f"Rx Sample Rate (sample pairs/pps interval) = {rx_sample_rate}")

# The sample rate in samples per second
print(f"Meta Transport Sample Rate (samples/s) = {meta_transport_sample_rate}")

### Pre-VCC Histogram

In [ ]:
device = f"talondx-{talon}/histogram/e_pre_vcc"
timeout = 3000  # ms
client = HistogramClient(device, timeout)

# data is an array of [pol_x_data, pol_y_data], or None if success is false.
success, data = client.capture()
print(f"Histogram capture successful: {success}")

if success:
    plot_histogram(device, data)

### Packet Stream Repair

In [ ]:
# Each board has 4 packet stream repair devices packet_stream_repair_[0-3]
for i in range(4):
    device = f"talondx-{talon}/packetstreamrepair/packet_stream_repair_{i}"
    dp = DeviceProxy(device)

    packet_rate = dp.read_attribute("packet_rate").value
    # Read the number of packets received per second.
    # Wait 1 second to read the rx_packet_rate, in case it needs some time to start flowing
    wait_time_s = 1
    if dp.read_attribute("rx_packet_rate").value == 0:
        sleep(wait_time_s)
    rx_packet_rate = dp.read_attribute("rx_packet_rate").value
    los_seconds = dp.read_attribute("los_seconds").value
    link_failure = dp.read_attribute("link_failure").value
    packet_loss_count = dp.read_attribute("packet_loss_count").value
    packet_loss = dp.read_attribute("packet_loss").value
    packet_error_count = dp.read_attribute("packet_error_count").value
    packet_error = dp.read_attribute("packet_error").value

    print(
        f"{device}:\n\tpacket_rate = {packet_rate}\n\trx_packet_rate = {rx_packet_rate}\n\tlos_seconds = {los_seconds}\n\tlink_failure = {link_failure}\n\tpacket_loss_count = {packet_loss_count}\n\tpacket_loss = {packet_loss}\n\tpacket_error_count = {packet_error_count}\n\tpacket_error = {packet_error}\n"
    )

### Post-VCC Histogram

In [ ]:
# Each board has 4 Post-VCC devices e_post_vcc_[0-3]
device = f"talondx-{talon}/histogram/e_post_vcc_1"
timeout = 3000  # ms
client = HistogramClient(device, timeout)

# data is an array of [pol_x_data, pol_y_data], or None if success is false.
success, data = client.capture()
print(f"Histogram capture successful: {success}")

if success:
    plot_histogram(device, data)

### Post-Resampler Delay Tracker (RDT) Histogram

In [ ]:
# Each board has 4 Post-RDT devices e_post_rdt_[0-3]
device = f"talondx-{talon}/histogram/e_post_rdt_1"
timeout = 3000  # ms
client = HistogramClient(device, timeout)

# data is an array of [pol_x_data, pol_y_data], or None if success is false.
success, data = client.capture()
print(f"Histogram capture successful: {success}")

if success:
    plot_histogram(device, data)

### Post-16K Channelizer Histogram

In [ ]:
# Each board has 4 Post-ch16k devices e_post_ch16k_[0-3]
device = f"talondx-{talon}/histogram/e_post_ch16k_1"
timeout = 3000  # ms
client = HistogramClient(device, timeout)

# data is an array of [pol_x_data, pol_y_data], or None if success is false.
success, data = client.capture()
print(f"Histogram capture successful: {success}")

if success:
    plot_histogram(device, data)

### DDR4 Corner Turner (DCT)

In [ ]:
device = f"talondx-{talon}/dct/dct"
dp = DeviceProxy(device)

number_of_channels = dp.read_attribute("number_of_channels").value
number_of_inputs = dp.read_attribute("number_of_inputs").value
number_of_samples = dp.read_attribute("number_of_samples").value
start_read_ts_lo = dp.read_attribute("start_read_timestamp_lo").value
start_read_ts_hi = dp.read_attribute("start_read_timestamp_hi").value
start_read_timestamp = (start_read_ts_hi << 32) + start_read_ts_lo

first_write_timestamps = []
first_write_ts_lo = dp.read_attribute("first_write_timestamp_lo").value
first_write_ts_hi = dp.read_attribute("first_write_timestamp_hi").value
for lo, hi in zip(first_write_ts_lo, first_write_ts_hi):
    ts = (hi << 32) + lo
    # need to convert np.int64 to int for safe_dump
    first_write_timestamps.append(int(ts))

print(f"Number of channels = {number_of_channels}")
print(f"Number of inputs = {number_of_inputs}")
print(f"Number of samples = {number_of_samples}")
print(f"Start read timestamp = {start_read_timestamp}")
print(f"First write timestamps = {first_write_timestamps}")

### 100gbe Ethernet 

In [ ]:
# 100g_eth_0 is the input from SPFRx. 100g_eth_1 is the output to the cluster where SDP is
device = f"talondx-{talon}/ska-talondx-100-gigabit-ethernet/100g_eth_1"
dp = DeviceProxy(device)

# Number of transmitted bytes in frames with no FCS, undersized, oversized, or payload length errors
# The number should continuously be incrementing when outputting visibilities
txFrameOctetsOK = dp.read_attribute("TxFrameOctetsOK").value
print(f"t = 0: {txFrameOctetsOK}")
sleep(1)
txFrameOctetsOK = dp.read_attribute("TxFrameOctetsOK").value
print(f"t = 1: {txFrameOctetsOK}")
sleep(1)
txFrameOctetsOK = dp.read_attribute("TxFrameOctetsOK").value
print(f"t = 2: {txFrameOctetsOK}")